## Hand Sign Detection
By : Reynara Ezra Pratama

### Import Library

In [3]:
import tensorflow as tf
print(tf.__version__)

2.6.0


In [4]:
!pip install ipython-autotime
%load_ext autotime

time: 133 µs (started: 2021-10-31 11:25:51 +00:00)


In [6]:
import zipfile, os

local_zip = '/content/drive/MyDrive/Data/dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/Hand Sign Detection/Dataset')
zip_ref.close()

time: 6.17 s (started: 2021-10-31 11:26:03 +00:00)


In [7]:
print(os.listdir('/tmp/Hand Sign Detection/Dataset'))

['Sign Language for Numbers']
time: 4.57 ms (started: 2021-10-31 11:26:13 +00:00)


In [27]:
type_class = os.listdir('/tmp/Hand Sign Detection/Dataset/Sign Language for Numbers')
type_class = sorted(type_class)
print(type_class)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'unknown']
time: 3.67 ms (started: 2021-10-31 11:37:18 +00:00)


In [36]:
total_data = 0
for i in type_class:
    directory = os.listdir(f'/tmp/Hand Sign Detection/Dataset/Sign Language for Numbers/{i}')
    print(f'Number of {i} Images : {len(directory)}')
    total_data += len(directory)

print(f'\nTotal Images :{total_data}')

Number of 0 Images : 1500
Number of 1 Images : 1500
Number of 2 Images : 1500
Number of 3 Images : 1500
Number of 4 Images : 1500
Number of 5 Images : 1500
Number of 6 Images : 1500
Number of 7 Images : 1500
Number of 8 Images : 1500
Number of 9 Images : 1500
Number of unknown Images : 1500

Total Images :16500
time: 28.7 ms (started: 2021-10-31 11:42:07 +00:00)
